In [15]:
import matplotlib.pyplot as plt
import torch
import torchvision
import seaborn as sns
import pandas as pd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torch import optim
import torch.nn as nn 
from torch.utils.data import Subset
import timm

In [16]:


def get_indices(data, x, y):
    return [i for i in range(len(data)) if (data[i][1] == x) or (data[i][1] == y)]


def get_train_val_subsets(data, subset, train_percentage=0.9):
    n_train = int(len(subset) * train_percentage)
    train = Subset(data, subset[:n_train])
    test = Subset(data, subset[n_train:])
    return train, test


def train_loop(n_epochs, batch_size, test_bsize, optimizer, model, loss_fn, train_loader, val_loader):
    for epoch in range(0, n_epochs):
        print(f'    EPOCH {epoch} ')
        mu_loss_train = 0

        model.train()
        for n, (x, y) in enumerate(train_loader):
            x = x.view(batch_size, -1).cuda()
            x_hat = model(x)

            loss_train = loss_fn(x_hat, x)
            loss_train.backward()
            optimizer.step()
            optimizer.zero_grad()

            mu_loss_train += loss_train.item()

        train_loss = mu_loss_train / (batch_size * len(train_loader))
        print(f'        muE:{train_loss:.4f}')

        #if (epoch == 0) or ((epoch % 2) == 0):
        #    # Validation
        #    with torch.no_grad():
        #        mu_loss_val = 0
        #        model.eval()
        #        for x_val, y_val in val_loader:
        #            x_val = x_val.view(test_bsize, -1).cuda()
        #            x_hat_val = model(x_val)
        #            mu_loss_val += loss_fn(x_hat_val, x_val).item()

        #    val_loss = mu_loss_val / (test_bsize * len(val_loader))
        #    print(f'        muE_val:{val_loss:.4f}')



In [17]:
# Download data
train_cifar_data = datasets.CIFAR10('~/data', train=True, download=False, transform=transforms.ToTensor())
test_cifar_data = datasets.CIFAR10('~/data', train=False, download=False, transform=transforms.ToTensor())
len(train_cifar_data), len(test_cifar_data)

(50000, 10000)

In [18]:
# Extract example indices of two classes
d12_idx = get_indices(train_cifar_data, 0, 1)

# Split indices in D1 and D2 
d1_idx = d12_idx[:5000]
d2_idx = d12_idx[5000:]

# Creates D1,D2 train and validation
d1_tr, d1_val = Subset(train_cifar_data, d1_idx[:4500]), Subset(train_cifar_data, d1_idx[4500:])
d2_tr, d2_val = Subset(train_cifar_data, d2_idx[:4500]), Subset(train_cifar_data, d2_idx[4500:])

In [19]:
# Model definition
model =  timm.create_model('resnet18', pretrained=False, num_classes=2)
model.to('cpu')
model.eval()

# Define loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer = optim.SGD(model.parameters(), lr=1e-4)
loss_fn = nn.BCELoss()


# Train Loop

In [23]:
batch_size = 64
test_bsize = 16

train_loader, val_loader = DataLoader(d1_tr, batch_size=batch_size, drop_last=False), DataLoader(d1_val, batch_size=test_bsize, drop_last=False)


for epoch in range(0, 5):
    print(f'    EPOCH {epoch} ')
    mu_loss_train = 0

    model.train()
    for n, (x, y) in enumerate(train_loader):
        #x = x.view(batch_size, -1).cuda()
        #x = x.cuda()
        x_hat = model(x)

        loss_train = loss_fn(x_hat, x)
        loss_train.backward()
        optimizer.step()
        optimizer.zero_grad()

        mu_loss_train += loss_train.item()

    train_loss = mu_loss_train / (batch_size * len(train_loader))
    print(f'        muE:{train_loss:.4f}')

    if (epoch == 0) or ((epoch % 2) == 0):
            # Validation
            with torch.no_grad():
                mu_loss_val = 0
                model.eval()
                for x_val, y_val in val_loader:
                    x_val = x_val.view(test_bsize, -1).cuda()
                    x_hat_val = model(x_val)
                    mu_loss_val += loss_fn(x_hat_val, x_val).item()

            val_loss = mu_loss_val / (test_bsize * len(val_loader))
            print(f'        muE_val:{val_loss:.4f}')

    EPOCH 0 


/home/lakj/miniconda3/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


ValueError: Using a target size (torch.Size([64, 3, 32, 32])) that is different to the input size (torch.Size([64, 2])) is deprecated. Please ensure they have the same size.

NameError: name 'sbs_tr_one' is not defined

333.3333333333333